TUTORIAL 1 : NVT Lennard-Jones fluid
====================================

Original authors: Andrey V. Brukhno (andrey.brukhno{at}stfc.ac.uk), James Grant (r.j.grant{at}bath.ac.uk), and John Purton (john.purton{at}stfc.ac.uk)

Modified to use dlmontepython by Joe Manning (joseph.manning{at}manchester.ac.uk)

MC method in outline
---------------------

In this tutorial we will perform the most straightforward Monte Carlo (MC) simulation - a simulation of a Lennard-Jones fluid in the canonical (NVT)
ensemble. The triplet **NVT** signifies that number of particles, volume (hence, density) and temperature are kept constant during the entire length of
this type of simulation.

In this case only the particles are allowed to move, and a random walk is constructed following the approach of Metropolis *et al.*.
The Metropolis algorithm iterates the following three steps that together constitute a single MC move (or attempt):

1. Select a particle at random and calculate its energy $U(\mathbf{r}_1)$.
2. Displace the particle randomly to a *trial* position and calculate its new energy $U(\mathbf{r}_2)$.
3. Accept or reject the particle displacement according to the Metropolis acceptance rule:

$$ P_{\mathrm{acc}}(\mathbf{r}_1 \rightarrow \mathbf{r}_2) = \min(1, \exp \{- \beta [U(\mathbf{r}_2) - U(\mathbf{r}_1)] \} ) $$


How do we achieve this?
-----------------------

**Ensure detailed balance**: it is crucial to ensure a truly random (uniform) selection of trial particle positions! Otherwise an unwanted, hidden bias can be introduced into the MC sampling procedure, which would invalidate the obtained statistics.

The common way to move (translate) a particle is to generate a random displacement vector $\Delta\mathbf{r}$ bound within a cube with a given side legnth $\Delta_{max}$ (the so-called *displacement parameter*) and add it to the current position of the particle under trial.


Then, pick up a random number between 0 and 1 and accept or reject the move as illustrated below:

<img src="images/Metropolis-acceptance.jpg" alt="Demonstration of the Metropolis algorithm" class="bg-primary" width="500px"> 

DL_MONTE Input Files
--------------------

The purpose of this tutorial is to introduce the basic file formats that are used by DL_MONTE and then run an example MC simulation. DL_MONTE requires at least 3 standard input files (similar to DL_POLY):

 FIELD -- units, atoms, molecules, topology and force-field specs (fixed format)
 
 CONFIG -- initial configuration for the system, including cell vectors (fixed format)
 
 CONTROL -- simulation parameters: external conditions, options and flags (free format)

NOTE: The file names are *fixed*, i.e. only these file names are recognised by DL_MONTE. In some more intricate cases other input files might be needed (e.g. for restarting a simulation for continuation).

We will introduce these files and discuss each of them in turn. Full details of their structure, formats and default values can be found in the [DL_MONTE manual](https://dl_monte.gitlab.io/dl_monte_manual/).

In this tutorial, we will go through the basics of producing each of these fiules using DL_MONTE's python wrapper [dlmontepython](https://gitlab.com/dl_monte/dlmontepython)

NOTE: The first line in all the three files is an arbitrary *title* which can contain up to 80 characters (any symbol over that limit will be ignored by DL_MONTE). The titles can be different in each file. 


FIELD
-----

In DL_MONTE the system is abstracted into atoms and molecules: atoms are treated as point-like particles, and molecules are collections of atoms which can be moved collectively. This, along with a versatile selection of potential forms which can be combined into different force fields, allows for simulation of a wide range of systems comprised of possible combinations of free unconnected atoms (so-called atomic/ionic fields), and molecules possessing structure: both rigid and flexible.

The FIELD file defines all the atomic and molecular types present in the system, but also possibly the molecular structure (so-called topology: bonds, angles, etc). Finally, it also specifies the interatomic and (optional) external potentials. A detailed description of all the available force-field parameters and their interplay can be found in the DL_MONTE manual. 

The file structure is similar to that used by DL_POLY, but there are some important differences owing to the requirements for Grand-Canonical simulations (considered later). 

The FIELD file for our first tutorial is shown below:

```

   Lennard-Jones, 2.5*sigma cut-off, sigma = 1 Angstrom, epsilon = 1 internal unit = 10 J/N_A
   CUTOFF 2.5                      # cut-off radius (must be the 2-nd line - after the title)
   UNITS internal                  # energy unit (must be the 3-rd line - after CUTOFF)
   NCONFIGS   1                    # number of configurations in CONFIG file (must be the 4-th line)
   ATOM TYPES 1                    # number of atom types or elements (must be the 5-th line)
   LJ core 1.0  0.0                # mass and charge of the atom(s)
   MOLECULE TYPES 1                # number of molecular types
   lj                              # molecule name (case sensitive)
   MAXATOM 512                     # max number of atoms within current molecular type
   FINISH                          # closing the molecular type specs
   VDW 1                           # number of short-ranged Van der Waals potentials 
   LJ core  LJ core lj   1.0 1.0   # atom names and types for a VDW pair, LJ epsilon and sigma
   CLOSE                           # closing FIELD

```
The important points to note are: 

 the *cutoff = 2.5* for the short-ranged VDW interactions is in Angstroms 
 
 the energy *unit* used in this case is *internal = 10 J/mol* [or 0.01 kJ/mol], 
 
 the CONFIG file contains *NCONFIGS = 1* configuration.

 the only *ATOM* type present has the name *LJ* and the type *core*, having *mass = 1.0* and *charge = 0.0*. 
 
All the atoms in the system are contained within a single molecule called *lj*. The number of atoms in this molecule is restricted to 512 at most. Note that the CONFIG file cannot contain more than that number of atoms within that type of a molecule. As a matter of fact, for an NVT simulation where the number of particles cannot change, the CONFIG file *must* contain the total number of atoms that is specified in the FIELD file!


### Building a FIELD file
First, we import the specific dlmontepython modules we need. Primarily this is the `dlfield` module. Then we'll instantiate an exmpy FIELD object to check the defaults:

In [1]:
from dlmontepython.htk.sources import dlfield

test_field = dlfield.FIELD()
print(test_field)

Field file Header
CUTOFF 1.0
UNITS eV
NCONFIGS 1
ATOMS 0
MOLTYPES 0
FINISH
VDW 0
CLOSE


As we can see, there are a lot of parameters to change:
* Modify the global parameters (energy units, cutoff etc.) to match our test system
* Define atom- and molecule types
* Define intermolecular interactions between atoms in the system

We'll go through each in turn:

#### Defining global parameters

In [2]:
# Define global parameters
test_field.description = "Lennard-Jones, 2.5*sigma cut-off, sigma = 1 Angstrom, epsilon = 1 internal unit = 10 J/N_A"
test_field.cutoff = 2.5
test_field.units = 'internal'
print(test_field)

Lennard-Jones, 2.5*sigma cut-off, sigma = 1 Angstrom, epsilon = 1 internal unit = 10 J/N_A
CUTOFF 2.5
UNITS internal
NCONFIGS 1
ATOMS 0
MOLTYPES 0
FINISH
VDW 0
CLOSE


#### Defining specific atom- and molecule types

Atom and molecule type objects are defined in `dlfieldspecies`.
There are 4 objects defined within that module, but in this tutorial we're only using three - `AtomType`, `Atom`, and `MolType`. 
The other ones will come in handy later when we are defining multiatomic molecules with specific internal coordinates.

In [3]:
from dlmontepython.htk.sources import dlfieldspecies

#Defining a category of atoms that mgith be used in multiple places/moleculeswithin the same simulation
lj_atomtype = dlfieldspecies.AtomType(
    name = 'LJ',
    atype = 'core',
    mass = 1.,
    charge = 0.
)

# Defining a specific atom to be placed within a molecule - we'll use this later!
lj_atom = dlfieldspecies.Atom(
    name='LJ',
    atype = 'core',
    mass = 1.,
    chg = 0.,
    x=0.,
    y=0.,
    z=0.,
)

# Defining a molecule type, along the same lines as the AtomTpye above.
# In this case we just need to pass the maxatom command = it specifies that we don't need to consider internal coordinates
lj_moltype = dlfieldspecies.MolType(
    name='lj',
    nmaxatom=512
)

# Defining the molecule as a list of atoms with specific positions
lj_mol = dlfieldspecies.Molecule(
    name = 'lj'
)


In [4]:
# Now we add the two objects we're using to the test FIELD file
test_field.atomtypes.append(lj_atomtype)
test_field.moltypes.append(lj_moltype)
print(test_field)

Lennard-Jones, 2.5*sigma cut-off, sigma = 1 Angstrom, epsilon = 1 internal unit = 10 J/N_A
CUTOFF 2.5
UNITS internal
NCONFIGS 1
ATOMS 1
LJ core 1.0 0.0
MOLTYPES 1
lj
MAXATOM 512
FINISH
VDW 0
CLOSE


#### Adding interactinos to the FIELD file
Interactions are defined in `dlfieldinteraction`. 
There are a few different ones defined, but in this case we use traditional [Lennard Jones interactions](https://en.wikipedia.org/wiki/Lennard-Jones_potential) with the `InteractionLJ` object.

Once instantiated, we define which atomtypes are interacting based on a `dlfield.VDW` object.

In [5]:
from dlmontepython.htk.sources import dlinteraction

interaction = dlinteraction.InteractionLJ(
    epsilon=1.,
    sigma=1.
)

example_LJ = dlfield.VDW(
    atom1 = lj_atom,
    atom2 = lj_atom,
    interaction = interaction
)

test_field.vdw.append(example_LJ)

print(test_field)

Lennard-Jones, 2.5*sigma cut-off, sigma = 1 Angstrom, epsilon = 1 internal unit = 10 J/N_A
CUTOFF 2.5
UNITS internal
NCONFIGS 1
ATOMS 1
LJ core 1.0 0.0
MOLTYPES 1
lj
MAXATOM 512
FINISH
VDW 1
LJ core LJ core lj 1.0 1.0
CLOSE


#### Comparing against the template

Now that we've added in our atoms, let's do a final check to see how the FIELD file we generated compares against the given example. We'll do this with [`difflib`](https://docs.python.org/3/library/difflib.html#differ-example) which will print out a character-by-character summary of differences in the two objects.

In [6]:
from pprint import pprint
import difflib
D = difflib.Differ()

given = """Lennard-Jones, 2.5*sigma cut-off, sigma = 1 Angstrom, epsilon = 1 internal unit = 10 J/N_A
CUTOFF 2.5
UNITS internal
NCONFIGS 1
ATOM TYPES 1
LJ core 1.0 0.0
MOLECULE TYPES 1
lj
MAXATOM 512
FINISH
VDW 1
LJ core LJ core lj 1.0 1.0
CLOSE""".splitlines(keepends=False)

generated = str(test_field).splitlines(keepends=False)

result = list(D.compare(given, generated))
pprint(result)

['  Lennard-Jones, 2.5*sigma cut-off, sigma = 1 Angstrom, epsilon = 1 internal '
 'unit = 10 J/N_A',
 '  CUTOFF 2.5',
 '  UNITS internal',
 '  NCONFIGS 1',
 '- ATOM TYPES 1',
 '+ ATOMS 1',
 '  LJ core 1.0 0.0',
 '- MOLECULE TYPES 1',
 '?    ------\n',
 '+ MOLTYPES 1',
 '  lj',
 '  MAXATOM 512',
 '  FINISH',
 '  VDW 1',
 '  LJ core LJ core lj 1.0 1.0',
 '  CLOSE']


From the `Differ` output we can see the only difference in two two objects is the `ATOM TYPES` and `MOLECULE TYPES` keywords. A quick check of the relevant section in the [DL_MONTE manual](https://dl_monte.gitlab.io/dl_monte_manual/input.html#the-field-file:~:text=the%20latter%20are-,complete,-.) indicates that these differences will make no difference to the eventual outcome.

CONFIG
------

The CONFIG file contains the starting configuration specification. It defines the cell type and geometry, sets the cell vectors (in the matrix form), the actual and maximum numbers of molecules, atoms within each molecule, and also the coordinates of all the atoms. 

Below we illustrate the structure of the CONFIG file with a sample of the configuration provided for this tutorial.

```
   Exercise 1: LJ fluid in NVT ensemble
   0         0
   11.7452  0.00000  0.00000
   0.00000  11.7452  0.00000
   0.00000  0.00000  11.7452
   NUMMOL 1 1
   MOLECULE lj 512 512
   LJ core
   0 0 0
   LJ core
   0 0 0.125
   LJ core
   0 0 0.25
   LJ core
   0 0 0.375
```
   
In line 2 the first number is an integer switch operating in the same way as *levcfg* in DL_POLY. It should be set to zero if the CONFIG file only contains the particle coordinates (which is sufficient for starting an MC simulation). However, if non-zero, this flag allows DL_MONTE to read in CONFIG files originating from MD (DL_POLY) simulations and containing (x,y,z) components of the particle velocities (*levcfg = 1*) or velocities and forces (*levcfg = 2*) which would appear on the lines directly following the particle coordinates. Note, however, that DL_POLY CONFIG files cannot be used directly, without certain modifications, as input for DL_MONTE simulations. 

The second integer flag determines the convention used for the particle coordinates: 
* 0 for fractional representaion
* 1 for cubic
* 2 for orthorhombic
* 3 fr palleliped
These are followed by the cell matrix. 

In our case the CONFIG file contains fractional coordinates and, since the cell matrix is diagonal and all its diagonal elements are equal, the
simulation cell is cubic.

The line starting with *NUMMOL* keyword specifies the *total number of molecules present*, followed by the *maximum number of molecules allowed for
each molecular type* defined in the FIELD file. In this case we have only one molecular type *lj*, and there is only one molecule of this type in the
CONFIG file. The molecule comprises 512 atoms (LJ particles) which equals the maximum number. 

Finally, the atoms' specs are read in. The atom data follows and for each particle, its name (usually chemical symbol) and type (core, semi and metal
that can be abbreviate to c, s, and m) are specified.  
On the next line these are followed by the coordinates in a format consistent with that given on line 2.  

<img src="images/tutorial1-CONFIG1.png" alt="Visualisation of the initial configuration" class="bg-primary" width="450px"> 

### Building a CONFIG file with dlmontepython
As with the FIELD file, we build the CONFIG file from a speciifc module in `dlmontepython.htk.sources`. These two files are closely linked - the simulation will only run if the initial configuration is valid given the specific force field parameters and _vice versa_.

First, we instantiate the `CONFIG` file:

In [7]:
from dlmontepython.htk.sources import dlconfig

cell = [
    [11.7452, 0,0],
    [0,11.7452,0],
    [0,0,11.7452]
]

test_config = dlconfig.CONFIG(
    title="Exercise 1: LJ fluid in NVT ensemble",
    level=0,
    dlformat=0,
    vcell=cell,

)
print(test_config)

Exercise 1: LJ fluid in NVT ensemble
0 0
11.7452 0 0
0 11.7452 0
0 0 11.7452
NUMMOL 


Then we need to updatye our `Molecule` object from earlier with an 8x8x8 grid of LJ atoms. first we'll generate the intended positions with `itertools.product`, then we'll use the `lj_mol.add_atom` method to add a new 

In [8]:
def _atom_stringify(self):
    output = []
    for i in self:
        output.append(
            '{0} {1}\n {2} {3} {4} 0'.format(
                i.name,
                i.type,
                i.rpos[0],
                i.rpos[1],
                i.rpos[2]
            )
        )
    return output

def to_dict(self):
    """
    returns a dictionary representation for inputting into a CONFIG object
    """
    output = {}
    output["name"] = self.name
    output["natom"] = len(self.atoms)
    output["atoms"] =  _atom_stringify(self.atoms)

    return output

In [9]:
from itertools import product
from copy import deepcopy


fractional_positions = [
    0,
    0.125,
    0.25,
    0.375,
    0.5,
    0.625,
    0.75,
    0.875
]

positions = product(fractional_positions, repeat=3)
for pos in positions:
    working = deepcopy(lj_atom)
    working.set_position(pos[0], pos[1], pos[2])
    lj_mol.add_atom(working)

test_config.nummol = [1,1]

test_config.molecules = [to_dict(lj_mol)]
print(str(test_config)[:220])

Exercise 1: LJ fluid in NVT ensemble
0 0
11.7452 0 0
0 11.7452 0
0 0 11.7452
NUMMOL 1 1
MOLECULE lj 512
LJ core
 0.0, 0.0, 0.0 0
LJ core
 0.0, 0.0, 0.125 0
LJ core
 0.0, 0.0, 0.25 0
LJ core
 0.0, 0.0, 0.375 0
LJ core
 0.


As you can see from the top of the file, we've now got the right number of atoms in the CONFIG object and their positions seems to be uniformly distributed.

CONTROL
-------
 
The CONTROL provides directives to DL_MONTE how to undertake the calculations and switches on or off functionality. The CONTROL file in this example is:

```
   NVT simulation of Lennard-Jones fluid
   #use ortho
   finish
   seeds 12 34 56 78               # Seed RNG seeds explicitly to the default
   nbrlist auto                    # Use a neighbour list to speed up energy calculations
   maxnonbondnbrs 512              # Maximum number of neighbours in neighbour list
   temperature     1.4283461511745 # T* = 1.1876 = T(K) * BOLTZMAN (see constants_module.f90)
   steps          10000            # Number of moves to perform in simulation
   equilibration    0              # Equilibration period: statistics are gathered after this period
   print           1000            # Print statistics every 'print' moves
   stack           1000            # Size of blocks for block averaging to obtain statistics
   sample coord   10000            # How often to print configurations to ARCHIVE.000
   revconformat dlmonte            # REVCON file is in DL_MONTE CONFIG format
   archiveformat dlpoly4           # ARCHIVE.000/HISTORY.000/TRAJECTORY.000 format 
                                   # In this case: HISTORY.000 in DLPOLY4 style
   move atom 1 100                 # Move atoms with a weight of 100
   LJ core
   start
```

The first line is the title and the second contains the keyword *finish*. 
We will see later that there a number of directives in DL_MONTE that can only occur before this directive which *must* be present in the CONTROL file.

*Seeds* followed by a series of 4 integers provides a reproducible seed, otheriwse *ranseed* generates a random seed from the system clock at initialisation.

The diretives *nbrlist* and *maxnonbondnbrs* control the size and administration of the neighbourlist used by DL_MONTE to optimise performance, and are explained further in one of the exercises.

*Temperature* (K) is self explanatory while *steps* is the length of the simulation in attempted  moves.
NOTE: Providing that LJ epsilon = 1.0 in FIELD, the reduced LJ temperature $T* = T(K)*R$, where R = BOLTZMAN (see constants_module.f90 or $\beta = 1/RT(K)$ in OUTPUT.000). In general, $T* = RT(K)/\epsilon = 1/(\epsilon*\beta)$, where epsilon is in *internal energy units*.

*Equilibration* etc control the detail of how data is output from DL_MONTE.

The *revconformat dlmonte* instruction describes the format of the output file REVCON.000 which contains the final configuration of the simulation and can be used for continuing a simulation, *dlpoly2*, *dlpoly4* are other options.

*archiveformat dlpoly4* describes the format of the trajectory file here it will be HISTORY.000, equivalent of the HISTORY in DLPOLY4.  The *dlpoly{2,4}* formats are readable by common visualisation packages such as vmd.  Full options are detailed in the manual.

The directive *move atoms* is where we begin to touch on the fundamental control of the simulation.
The key feature here is that DL_MONTE will not do anything unless told to do so (N.B. While this gives DL_MONTE great flexibility it means also means that it may be possible to ask DL_MONTE to perform ill-defined calculations). 
In this simple *NVT* ensemble only the particles move, thus *move atoms* tells DL_MONTE to move *1* atom type, with a weight of  *100*. 
The line(s) following this detail the atom and type.

Finally the *start* directive ends the *CONTROL* file and instructs DL_MONTE to start the simulation.


### Setting up a CONTROL file with dlmontepython
As with before, we can setup a CONTROL file with the appropriate module in `dlmontepython.htk.sources`:

In [10]:
from dlmontepython.htk.sources import dlcontrol

test_control = dlcontrol.CONTROL(
    title="NVT simulation of Lennard-Jones fluid"
)
print(test_control)

NVT simulation of Lennard-Jones fluid
None
None


This empty CONTROL object has no simulaiton parameters in it yet - we provide these through the `UseBlock` and `MainBlock` objects. 
The `UseBlock` object defines some overarching aspects of the simulation which will be explained in later tutorials. 
For this tutorial, we will define just about everything from the `MainBlock` object.

In [11]:
test_use = dlcontrol.UseBlock()
print(test_use)
test_main = dlcontrol.MainBlock()
print(test_main)

finish use-block
start simulation


By adding these in to the `CONTROL` object, we can set the simulation parameters through `CONTROL`'s various class methods.

In [12]:
test_control.use_block = test_use
test_control.main_block = test_main
print(test_control)

NVT simulation of Lennard-Jones fluid
finish use-block
start simulation


In [13]:
test_control.set_seeds(
    i = 12,
    j = 34,
    k = 56,
    l = 78
)
test_control.main_block.statements['nbrlist'] = "auto"
test_control.main_block.statements['maxnonbondnbrs'] = 512
test_control.set_temperature(1.4283461511745)
test_control.set_steps(10000)
test_control.set_equilibration(0)
test_control.set_print(1000)
test_control.set_stack(1000)
test_control.main_block.samples['coord'] = {'0': 10000}
test_control.main_block.statements["revconformat"] = "dlmonte"
test_control.main_block.statements["archiveformat"] = "dlmonte"

print(test_control)

NVT simulation of Lennard-Jones fluid
finish use-block
seeds 12 34 56 78
nbrlist auto
maxnonbondnbrs 512
temperature 1.4283461511745
steps 10000
equilibration 0
print 1000
stack 1000
revconformat dlmonte
archiveformat dlmonte
sample coord 10000
start simulation


Finally, we can add in the Monte Carlo move as an MCMove object from `dlmontepython.htk.sources.dlmove`

In [14]:
from dlmontepython.htk.sources import dlmove

test_move =dlmove.AtomMove(
    pfreq=100,
    movers = [{"id": 'LJ core'}]
)
print(test_move)
test_control.main_block.moves.append(test_move)


move atom 1 100
LJ core


In [15]:
print(test_control)

NVT simulation of Lennard-Jones fluid
finish use-block
seeds 12 34 56 78
nbrlist auto
maxnonbondnbrs 512
temperature 1.4283461511745
steps 10000
equilibration 0
print 1000
stack 1000
revconformat dlmonte
archiveformat dlmonte
sample coord 10000
move atom 1 100
LJ core
start simulation


Finally, the `CONTROL` object appears complete. Let's just check it against the provided example:

In [16]:
given = """NVT simulation of Lennard-Jones fluid
finish
seeds 12 34 56 78
nbrlist auto
maxnonbondnbrs 512
temperature 1.4283461511745
steps 10000
equilibration 0
print 1000
stack 1000
sample coord 10000
revconformat dlmonte
archiveformat dlpoly4

move atom 1 100
LJ core
start""".splitlines(keepends=False)

generated = str(test_control).splitlines(keepends=False)

result = list(D.compare(given, generated))
pprint(result)

['  NVT simulation of Lennard-Jones fluid',
 '- finish',
 '+ finish use-block',
 '  seeds 12 34 56 78',
 '  nbrlist auto',
 '  maxnonbondnbrs 512',
 '  temperature 1.4283461511745',
 '  steps 10000',
 '  equilibration 0',
 '  print 1000',
 '  stack 1000',
 '+ revconformat dlmonte',
 '+ archiveformat dlmonte',
 '  sample coord 10000',
 '- revconformat dlmonte',
 '- archiveformat dlpoly4',
 '- ',
 '  move atom 1 100',
 '  LJ core',
 '- start',
 '+ start simulation']


## Setting up a simulation
Now that we have our `CONTROL`, `CONFIG`, and `FIELD` objects defined, let's get them printed out to a directory to test they run!

In [17]:
import os
try:
    os.mkdir('./tutorial_1_simulation')
except FileExistsError:
    pass
with open('./tutorial_1_simulation/CONTROL', 'w') as f:
    f.write(str(test_control))
    
with open('./tutorial_1_simulation/FIELD', 'w') as f:
    f.write(str(test_field))
    
with open('./tutorial_1_simulation/CONFIG', 'w') as f:
    f.write(str(test_config))
    

To test the simulation you'll need to open a terminal, navigate to `tutorial_1_simulation`, and run a compiled DL_MONTE executable. DL_MONTE will read in your files from the directory your terminal is in, simulate, and output to the same directory. Once done, head to tutorial 2!